# Calculating joint statistics from resulting AJIVE matrices 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
a_name = 'TCGA-BRCA'
b_name = 'CCLE'

a_path = '/home/baprice/Projects/00014_geneJIVE/data/prejive/tcgabrca_prejive.csv'
b_path = '/home/baprice/Projects/00014_geneJIVE/data/prejive/ccle_nohaem_prejive.csv'

a_joint_path = '/home/baprice/Projects/00014_geneJIVE/data/gj_tcgabrca_ccle/gj_tcgabrca_ccle_275+150/gj_tcgabrca_ccle_275+150_tcgabrca_joint.csv'
b_joint_path = '/home/baprice/Projects/00014_geneJIVE/data/gj_tcgabrca_ccle/gj_tcgabrca_ccle_275+150/gj_tcgabrca_ccle_275+150_ccle_joint.csv'

a_indiv_path = '/home/baprice/Projects/00014_geneJIVE/data/gj_tcgabrca_ccle/gj_tcgabrca_ccle_275+150/gj_tcgabrca_ccle_275+150_tcgabrca_individual.csv'
b_indiv_path = '/home/baprice/Projects/00014_geneJIVE/data/gj_tcgabrca_ccle/gj_tcgabrca_ccle_275+150/gj_tcgabrca_ccle_275+150_ccle_individual.csv'

a = pd.read_csv(a_path, index_col=0)
b = pd.read_csv(b_path, index_col=0)
a_joint = pd.read_csv(a_joint_path, index_col=0)
b_joint = pd.read_csv(b_joint_path, index_col=0)
a_indiv = pd.read_csv(a_indiv_path, index_col=0)
b_indiv = pd.read_csv(b_indiv_path, index_col=0)

In [ ]:
def distance(x1, y1, a=-1, b=1, c=0):  
    return abs((a * x1 + b * y1 + c)) / (np.sqrt(a * a + b * b)) 

def joint_statistics(joint, individual):
    s = 0.05
    return np.var(joint, axis=1).clip(lower=s)-(np.var(individual, axis=1).clip(lower=s))

def permute_expected_statistics(joint, individual, n_perm=20):
    joined = joint.join(individual, lsuffix='_J', rsuffix='_I')
    perms = []
    sig_counts = []
    for p in range(n_perm):
        col_perm = np.random.permutation(joined.columns.tolist())
        j_perm = joined[col_perm[:len(col_perm)//2]]
        i_perm = joined[col_perm[len(col_perm)//2:]]
        exp_stat = np.sort(joint_statistics(j_perm, i_perm))[::-1]
        perms.append(exp_stat)
    
    return pd.DataFrame(perms)

def qvalueTable(observed, permutations, thresholds=2000, decimals=4):
    obs_sigs = []
    perm_sigs = []
    fdrs = []
    qs = []
    for q in np.round(np.linspace(0, np.max([np.abs(observed.min()),np.abs(observed.max())]), num=thresholds), decimals=decimals):
        obs_sig = (np.abs(obs) > q).sum()
        perm_sig = (np.abs(permutations) > q).to_numpy().sum()
        fdr = perm_sig/(obs_sig+perm_sig)
        obs_sigs.append(obs_sig)
        perm_sigs.append(perm_sig)
        fdrs.append(fdr)
        qs.append(q)
        
    return_df = pd.DataFrame([qs, obs_sigs, perm_sigs, fdrs], index=['threshold','observed','expected','fdr']).T
    
    return return_df

def jstat_maplot(joint, individual):
    m = joint_statistics(joint, individual)
    a = average_j_stat(joint, individual)
    
    sns.set_context('notebook')
    plt.figure(figsize=[18,10])
    p = sns.scatterplot(y=m, x=a, alpha=0.35)
    ymin, ymax = p.axes.get_ylim()
    abs_ymax = max(abs(ymin), abs(ymax))
    plt.ylim([-abs_ymax, abs_ymax])
    plt.ylabel(r'$\log{\frac{\sigma_J}{\sigma_I}}$', size=30, rotation=0, labelpad=50)
    plt.xlabel(r'$\frac{1}{2}\log({\sigma_J\sigma_I})$', size=30, rotation=0)
    plt.axhline(0, color='k', zorder=1)
    
def average_j_stat(joint, individual):
    return 0.5 * np.log(np.std(joint, axis=1)*np.std(individual, axis=1))

In [ ]:
a_obs_j = pd.Series(joint_statistics(a_joint, a_indiv))
a_expect_j_perms = permute_expected_statistics(a_joint, a_indiv, n_perm=200)
a_expect_j = pd.DataFrame(a_expect_j_perms).mean()
a_expect_j.index = a_joint.index
a_obs_j_sort = a_obs_j.sort_values(ascending=False)
a_stat_df = pd.DataFrame([np.sort(a_obs_j)[::-1], a_expect_j], index=['Observed','Expected'], columns=a_obs_j_sort.index).T

b_obs_j = pd.Series(joint_statistics(b_joint, b_indiv))
b_obs_j_sort = b_obs_j.sort_values(ascending=False)
b_expect_j_perms = permute_expected_statistics(b_joint, b_indiv, n_perm=200)
b_expect_j = pd.DataFrame(b_expect_j_perms).mean()
b_expect_j.index = b_joint.index
b_stat_df = pd.DataFrame([np.sort(b_obs_j)[::-1], b_expect_j], index=['Observed','Expected'], columns=b_obs_j_sort.index).T

jstats_df = a_stat_df.join(b_stat_df, lsuffix='_'+str(a_name), rsuffix='_'+str(b_name))

In [ ]:
jstats_df.head()